## Importando Bibliotecas e Carregando Bases de Dados

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import time
import unicodedata
from collections import defaultdict, Counter

from itertools import combinations
from ortools.sat.python import cp_model

# Definindo variáveis gerais
participants = pd.read_excel('Rodada_Negocios_Netweaving_27102025_3.xlsx', sheet_name='Participants')
must_together = pd.read_excel('Rodada_Negocios_Netweaving_27102025_3.xlsx', sheet_name='Must_Together')
must_avoid = pd.read_excel('Rodada_Negocios_Netweaving_27102025_3.xlsx', sheet_name='Must_Avoid')
ramo_alias = pd.read_csv('ramo_alias.csv')
affinity_overrides = pd.read_csv('affinity_overrides.csv')

## Script Otimização

In [ ]:
# === Otimizador de Rodada de Negócios (CP-SAT / OR-Tools) — versão com fixos robustos ===
# Entradas já carregadas: participants, must_together, must_avoid, ramo_alias, affinity_overrides

def _env_int(var, default):
    val = os.getenv(var)
    if val is None:
        return default
    try:
        return int(val)
    except ValueError:
        return default


ROUNDS = _env_int('NW_ROUNDS', 4)
TABLES = _env_int('NW_TABLES', 17)
SEATS_PER_TABLE = _env_int('NW_SEATS_PER_TABLE', 4)

W_TOGETHER = 5000
W_SIM = 10
W_PREF1 = 300
W_PREF2 = 180
W_PREF3 = 90
PREF_FILLED_BOOST = 1.5
W_OVER = 800
DEFAULT_SIM_SAME = 0.30
DEFAULT_SIM_DIFF = 0.55

MAX_TIME_S = _env_int('NW_MAX_TIME_S', 120)
N_WORKERS = _env_int('NW_N_WORKERS', 8)

def strip_accents(s: str) -> str:
    if pd.isna(s):
        return ""
    s = str(s).strip().lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))
    return s

alias_map = {}
if {'entrada','normalizado'}.issubset({c.lower() for c in ramo_alias.columns}):
    _entrada = [c for c in ramo_alias.columns if c.lower()=='entrada'][0]
    _normalizado = [c for c in ramo_alias.columns if c.lower()=='normalizado'][0]
    for _,row in ramo_alias.iterrows():
        k = strip_accents(row[_entrada])
        v = str(row[_normalizado]).strip().lower()
        if k:
            alias_map[k] = v

def alias_norm(x: str) -> str:
    k = strip_accents(x)
    return alias_map.get(k, k if k else "outros")

cols = {c.lower(): c for c in participants.columns}
getcol = lambda name: cols.get(name)

id_col       = getcol('id')
nome_col     = getcol('nome_pessoa') or getcol('nome') or getcol('pessoa') or getcol('nome completo')
empresa_col  = getcol('nome_empresa') or getcol('empresa')
ramo_col     = getcol('ramo')
pref1_col    = getcol('pref1_ramo')
pref2_col    = getcol('pref2_ramo')
pref3_col    = getcol('pref3_ramo')
fixo_col     = getcol('fixo')          # pode não existir
mesa_fixa_col= getcol('mesa_fixa') or getcol('mesa fixa') or getcol('mesa fixa ')

dfP = participants.copy()

# id
if id_col is None:
    dfP['id'] = np.arange(len(dfP))
    id_col = 'id'

# nome/empresa/ramo (obrigatórios)
if nome_col is None:
    raise ValueError("Coluna 'nome_pessoa' (ou equivalente) não encontrada em Participants.")
if empresa_col is None:
    raise ValueError("Coluna 'nome_empresa' (ou equivalente) não encontrada em Participants.")
if ramo_col is None:
    raise ValueError("Coluna 'ramo' não encontrada em Participants.")

# normaliza ramo e prefs
dfP['ramo_norm'] = dfP[ramo_col].astype(str).apply(alias_norm)
for colname, newname in [(pref1_col,'pref1_norm'), (pref2_col,'pref2_norm'), (pref3_col,'pref3_norm')]:
    if colname:
        dfP[newname] = dfP[colname].astype(str).apply(lambda x: alias_norm(x) if str(x).strip() != "" else "")
    else:
        dfP[newname] = ""

# mesa fixa: força fixo=1 quando houver mesa definida
if mesa_fixa_col is None:
    dfP['mesa_fixa'] = np.nan
    mesa_fixa_col = 'mesa_fixa'
else:
    dfP['mesa_fixa'] = pd.to_numeric(dfP[mesa_fixa_col], errors='coerce')

# Heurística de base 1 → base 0 (se todos valores válidos estiverem em 1..TABLES e não houver 0)
if dfP['mesa_fixa'].notna().any():
    mf_nonnull = dfP['mesa_fixa'].dropna()
    cond_one_based = (mf_nonnull.min() >= 1) and (mf_nonnull.max() <= TABLES) and (0 not in set(mf_nonnull.astype(int)))
    if cond_one_based:
        dfP.loc[dfP['mesa_fixa'].notna(), 'mesa_fixa'] = dfP.loc[dfP['mesa_fixa'].notna(), 'mesa_fixa'].astype(int) - 1

# fixo (coluna pode não existir; aceita números, bools, strings)
def _to_bool01(x):
    if pd.isna(x): return 0
    if isinstance(x, (int, np.integer)): return 1 if int(x)!=0 else 0
    if isinstance(x, float): 
        if np.isnan(x): return 0
        return 1 if int(round(x))!=0 else 0
    s = str(x).strip().lower()
    return 1 if s in {'1','true','t','sim','yes','y'} else 0

if fixo_col is None:
    dfP['fixo'] = 0
else:
    dfP['fixo'] = dfP[fixo_col].apply(_to_bool01)

# **Regra nova**: quem tem mesa_fixa preenchida vira fixo=1 (sobrepõe)
dfP.loc[dfP['mesa_fixa'].notna(), 'fixo'] = 1

# sanity básico
N = len(dfP)
total_slots = TABLES * SEATS_PER_TABLE
if N > total_slots:
    raise RuntimeError(
        f"Existem {N} participantes para apenas {total_slots} vagas por rodada. "
        "Aumente o número de mesas ou assentos antes de rodar o solver.")
if N != total_slots:
    print(
        f"[Aviso] Há {N} participantes, mas a configuração prevê {total_slots} lugares por rodada. "
        "Mesas serão mantidas com 4 participantes por rodada; haverá cadeiras ociosas.")

# mapeamentos
P_ids = list(dfP[id_col].tolist())
id_to_idx = {pid:i for i,pid in enumerate(P_ids)}

names    = dfP[nome_col].astype(str).tolist()
empresas = dfP[empresa_col].astype(str).tolist()
ind_of   = dfP['ramo_norm'].astype(str).tolist()
pref1    = dfP['pref1_norm'].astype(str).tolist()
pref2    = dfP['pref2_norm'].astype(str).tolist()
pref3    = dfP['pref3_norm'].astype(str).tolist()
has_pref = [(str(pref1[i]).strip()!="") or (str(pref2[i]).strip()!="") or (str(pref3[i]).strip()!="") for i in range(N)]

fixed_info = {}
for i in range(N):
    if dfP['fixo'].iloc[i] == 1 and pd.notna(dfP['mesa_fixa'].iloc[i]):
        mf = int(dfP['mesa_fixa'].iloc[i])
        if not (0 <= mf < TABLES):
            raise ValueError(f"Mesa fixa fora do range [0..{TABLES-1}] para {names[i]}: {mf}")
        fixed_info[i] = mf  # fixo por mesa

# -------------------------
# must_together / must_avoid -> índices
# -------------------------
def get_pair_df(df, ca='id_a', cb='id_b'):
    if df is None or df.empty:
        return []
    cols = {c.lower(): c for c in df.columns}
    A = cols.get(ca.lower(), list(df.columns)[0])
    B = cols.get(cb.lower(), list(df.columns)[1])
    out = []
    for _,row in df.iterrows():
        a = row[A]; b = row[B]
        if pd.isna(a) or pd.isna(b): continue
        a = id_to_idx.get(a, None); b = id_to_idx.get(b, None)
        if a is None or b is None or a == b: continue
        if a > b: a,b = b,a
        out.append((a,b))
    return sorted(set(out))

must_together_pairs = get_pair_df(must_together)
must_avoid_pairs    = get_pair_df(must_avoid)

def diagnostic_report():
    print("\n=== Diagnóstico das entradas ===")
    print(f"Participantes: {N}")
    print(
        f"Mesas configuradas: {TABLES} | Assentos/mesa: {SEATS_PER_TABLE} | "
        f"Vagas totais por rodada: {total_slots}")
    print(
        f"Pares must_together: {len(must_together_pairs)} | "
        f"pares must_avoid: {len(must_avoid_pairs)}")

    if fixed_info:
        print("- Participantes fixos por mesa:")
        fixed_by_table = defaultdict(list)
        for idx, mesa in fixed_info.items():
            fixed_by_table[mesa].append(idx)
        for mesa in sorted(fixed_by_table):
            nomes = ", ".join(names[i] for i in fixed_by_table[mesa])
            print(f"  Mesa {mesa}: {nomes}")
    else:
        print("- Nenhum participante possui mesa fixa.")

    limit_mt = ROUNDS * (SEATS_PER_TABLE - 1)
    mt_degree = Counter()
    for a, b in must_together_pairs:
        mt_degree[a] += 1
        mt_degree[b] += 1

    alertas = []
    for idx, qtd in mt_degree.items():
        if qtd > limit_mt and idx not in fixed_info:
            alertas.append(
                f"{names[idx]} precisa encontrar {qtd} parceiros must_together, acima do limite teórico de {limit_mt} em {ROUNDS} rodadas.")
        elif idx in fixed_info and qtd > limit_mt:
            alertas.append(
                f"{names[idx]} (fixo na mesa {fixed_info[idx]}) possui {qtd} parceiros must_together; mesa fixa comporta no máximo {limit_mt} convidados distintos.")

    if alertas:
        print("[Alerta] Possível gargalo nos must_together:")
        for msg in alertas:
            print("  - " + msg)
    else:
        print("- Nenhum gargalo teórico de must_together detectado.")

    if must_avoid_pairs:
        ma_degree = Counter()
        for a, b in must_avoid_pairs:
            ma_degree[a] += 1
            ma_degree[b] += 1
        mais_ma = sorted(ma_degree.items(), key=lambda x: x[1], reverse=True)[:5]
        if mais_ma:
            print("- Participantes com mais restrições must_avoid:")
            for idx, qtd in mais_ma:
                print(f"  {names[idx]}: evita {qtd} participante(s)")
    print("=== Fim do diagnóstico ===\n")

diagnostic_report()

# -------------------------
# Pré-checagens (fail-fast)
# -------------------------
# a) capacidade por mesa só com fixos
fix_count = Counter(fixed_info.values())
overcap = {t:c for t,c in fix_count.items() if c > SEATS_PER_TABLE}
if overcap:
    raise RuntimeError(f"Capacidade excedida com fixos: {overcap}")

# b) must_avoid com dois fixos mesma mesa
bad_avoid = []
for (a,b) in must_avoid_pairs:
    if (a in fixed_info) and (b in fixed_info) and (fixed_info[a]==fixed_info[b]):
        bad_avoid.append((P_ids[a], P_ids[b], fixed_info[a]))
if bad_avoid:
    raise RuntimeError(f"Par(es) must_avoid estão fixos na mesma mesa (inviável): {bad_avoid}")

# c) must_together com dois fixos em mesas diferentes
bad_mt = []
for (a,b) in must_together_pairs:
    if (a in fixed_info) and (b in fixed_info) and (fixed_info[a]!=fixed_info[b]):
        bad_mt.append((P_ids[a], P_ids[b], fixed_info[a], fixed_info[b]))
if bad_mt:
    raise RuntimeError(f"Par(es) must_together fixos em mesas distintas (inviável): {bad_mt}")

# -------------------------
# Afinidade (sim) por ramo
# -------------------------
def sim_lookup(ind_a, ind_b):
    ia = str(ind_a).strip().lower()
    ib = str(ind_b).strip().lower()
    if len(affinity_overrides) > 0:
        cols = {c.lower(): c for c in affinity_overrides.columns}
        ra = cols.get('ramo_a', list(affinity_overrides.columns)[0])
        rb = cols.get('ramo_b', list(affinity_overrides.columns)[1])
        rs = cols.get('sim',    list(affinity_overrides.columns)[2])
        mask = ((affinity_overrides[ra].str.strip().str.lower()==ia) & 
                (affinity_overrides[rb].str.strip().str.lower()==ib))
        val = affinity_overrides.loc[mask, rs]
        if len(val)==0:
            mask = ((affinity_overrides[ra].str.strip().str.lower()==ib) & 
                    (affinity_overrides[rb].str.strip().str.lower()==ia))
            val = affinity_overrides.loc[mask, rs]
        if len(val)>0:
            try:
                return float(val.iloc[0])
            except:
                pass
    return DEFAULT_SIM_SAME if ia == ib else DEFAULT_SIM_DIFF

sim100 = {}
inds_unique = sorted(set(ind_of))
for a in inds_unique:
    for b in inds_unique:
        sim100[(a,b)] = int(round(sim_lookup(a,b) * 100))

# -------------------------
# Score de preferência (direcional)
# -------------------------
def pref_dir_score(i_from, i_to):
    ramo_to = ind_of[i_to]
    s = 0.0
    if ramo_to and ramo_to == pref1[i_from]: s += W_PREF1
    if ramo_to and ramo_to == pref2[i_from]: s += W_PREF2
    if ramo_to and ramo_to == pref3[i_from]: s += W_PREF3
    if has_pref[i_from]:
        s *= PREF_FILLED_BOOST
    return int(round(s))

# -------------------------
# Modelo CP-SAT
# -------------------------
R = range(ROUNDS)
T = range(TABLES)
P = range(N)
model = cp_model.CpModel()

# X[i,r,t] = 1 se pessoa i está na mesa t na rodada r
X = {(i,r,t): model.NewBoolVar(f"X_{i}_{r}_{t}") for i in P for r in R for t in T}

# 1) Cada pessoa ocupa exatamente 1 mesa por rodada
for i in P:
    for r in R:
        model.Add(sum(X[i,r,t] for t in T) == 1)

# 2) Capacidade: 4 por mesa/rodada
for r in R:
    for t in T:
        model.Add(sum(X[i,r,t] for i in P) == SEATS_PER_TABLE)

# 3) Fixos: mesa fixa em todas as rodadas
for i, mf in fixed_info.items():
    for r in R:
        model.Add(X[i,r,mf] == 1)

# 3b) Não fixos não podem repetir mesa (cada mesa no máx. 1x para a pessoa)
for i in P:
    if i not in fixed_info:
        for t in T:
            model.Add(sum(X[i,r,t] for r in R) <= 1)

# 4) must_avoid
for (a,b) in must_avoid_pairs:
    for r in R:
        for t in T:
            model.Add(X[a,r,t] + X[b,r,t] <= 1)

# 5) Indicadores de encontro por par/rodada/mesa (somente a<b)
pairs = [(a,b) for (a,b) in combinations(P,2)]
B = {}
for (a,b) in pairs:
    for r in R:
        for t in T:
            B[a,b,r,t] = model.NewBoolVar(f"B_{a}_{b}_{r}_{t}")
            model.AddBoolAnd([X[a,r,t], X[b,r,t]]).OnlyEnforceIf(B[a,b,r,t])
            model.AddBoolOr([X[a,r,t].Not(), X[b,r,t].Not()]).OnlyEnforceIf(B[a,b,r,t].Not())

# 6) Pares não must_together: no máximo 1 encontro no evento
must_together_set = set(must_together_pairs)
for (a,b) in pairs:
    if (a,b) not in must_together_set:
        model.Add(sum(B[a,b,r,t] for r in R for t in T) <= 1)

# 7) Y[a,b] para must_together atendido
Y = {}
for (a,b) in must_together_pairs:
    Y[a,b] = model.NewBoolVar(f"Y_{a}_{b}")
    model.Add(Y[a,b] <= sum(B[a,b,r,t] for r in R for t in T))
    for r in R:
        for t in T:
            model.Add(Y[a,b] >= B[a,b,r,t])

# 8) Diversidade por mesa: penaliza cnt(ramo) > 1
inds_present = sorted(set(ind_of))
over_vars = []
for r in R:
    for t in T:
        for ind in inds_present:
            cnt = model.NewIntVar(0, SEATS_PER_TABLE, f"cnt_{ind}_r{r}_t{t}")
            inds_idx = [i for i in P if ind_of[i] == ind]
            if len(inds_idx) == 0:
                model.Add(cnt == 0)
            else:
                model.Add(cnt == sum(X[i,r,t] for i in inds_idx))
            over = model.NewIntVar(0, SEATS_PER_TABLE, f"over_{ind}_r{r}_t{t}")
            model.Add(over >= cnt - 1)
            model.Add(over >= 0)
            over_vars.append(over)

# -------------------------
# Função objetivo (minimização)
# -------------------------
obj_terms = []

# must_together (max)
for (a,b), yvar in Y.items():
    obj_terms.append((-W_TOGETHER, yvar))

# afinidade + preferências (max)
for (a,b) in pairs:
    s_sim  = W_SIM * sim100[(ind_of[a], ind_of[b])]
    s_pref = pref_dir_score(a, b) + pref_dir_score(b, a)
    s_pair = s_sim + s_pref
    if s_pair != 0:
        for r in R:
            for t in T:
                obj_terms.append((-s_pair, B[a,b,r,t]))

# diversidade (min)
for over in over_vars:
    obj_terms.append((W_OVER, over))

model.Minimize(sum(coef * var for (coef, var) in obj_terms))

# -------------------------
# Solver
# -------------------------
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = MAX_TIME_S
solver.parameters.num_search_workers = N_WORKERS

status = solver.Solve(model)
status_name = solver.StatusName(status)
obj_val = solver.ObjectiveValue() if status in (cp_model.OPTIMAL, cp_model.FEASIBLE) else None
bound = solver.BestObjectiveBound()
print(f"[Solver] Status: {status_name} | Objetivo: {obj_val} | Bound: {bound}")

if status == cp_model.INFEASIBLE:
    raise RuntimeError("Modelo infeasible (verifique pares must_avoid/must_together e distribuição de fixos).")
if status == cp_model.MODEL_INVALID:
    raise RuntimeError("Modelo inválido. Revise se todos os dados foram lidos corretamente.")
if status == cp_model.UNKNOWN:
    raise RuntimeError(
        f"Solver não encontrou solução dentro do limite de {MAX_TIME_S}s (status UNKNOWN). "
        "Considere aumentar NW_MAX_TIME_S ou simplificar as restrições.")

# -------------------------
# Extrai solução
# -------------------------
assign = {(r,t): [] for r in R for t in T}
for i in range(N):
    for r in R:
        for t in T:
            if solver.Value(X[i,r,t]) == 1:
                assign[(r,t)].append(i)

# DataFrame longo
rows = []
for r in R:
    for t in T:
        for i in assign[(r,t)]:
            rows.append({
                "rodada": r,
                "mesa": t,
                "id": P_ids[i],
                "nome_pessoa": names[i],
                "nome_empresa": empresas[i],
                "ramo_norm": ind_of[i],
                "pref1_norm": pref1[i],
                "pref2_norm": pref2[i],
                "pref3_norm": pref3[i],
                "fixo": int(dfP['fixo'].iloc[i]),
                "mesa_fixa": int(dfP['mesa_fixa'].iloc[i]) if pd.notna(dfP['mesa_fixa'].iloc[i]) else np.nan,
            })
schedule_df = pd.DataFrame(rows).sort_values(["rodada","mesa","nome_pessoa"]).reset_index(drop=True)

# Visual compacto
def to_wide(g):
    ps = list(g.sort_values("nome_pessoa")["nome_pessoa"])
    row = {"rodada": int(g["rodada"].iloc[0]), "mesa": int(g["mesa"].iloc[0])}
    for k, nm in enumerate(ps[:SEATS_PER_TABLE], start=1):
        row[f"p{k}"] = nm
    return pd.Series(row)

schedule_wide = (
    schedule_df.groupby(["rodada","mesa"], as_index=False)
    .apply(to_wide)
    .sort_values(["rodada","mesa"])
    .reset_index(drop=True)
)

# Relatórios rápidos
mt_att = []
for (a,b) in must_together_pairs:
    atendeu = 0; r_hit = None; t_hit = None
    for r in R:
        for t in T:
            if solver.Value(B[a,b,r,t]) == 1:
                atendeu = 1; r_hit = r; t_hit = t; break
        if atendeu: break
    mt_att.append({"id_a": P_ids[a], "nome_a": names[a], "id_b": P_ids[b], "nome_b": names[b],
                   "atendido": atendeu, "rodada": r_hit, "mesa": t_hit})
must_together_report = pd.DataFrame(mt_att)

ma_viol = []
for (a,b) in must_avoid_pairs:
    for r in R:
        for t in T:
            if solver.Value(B[a,b,r,t]) == 1:
                ma_viol.append({"id_a": P_ids[a], "nome_a": names[a], "id_b": P_ids[b], "nome_b": names[b],
                                "rodada": r, "mesa": t})
must_avoid_violations = pd.DataFrame(ma_viol)

# Checagem extra: fixos realmente permaneceram na mesma mesa (e na mesa certa)
fixed_check = []
for i, mf in fixed_info.items():
    mesas_i = sorted(schedule_df.query("id == @P_ids[i]")["mesa"].unique().tolist())
    fixed_check.append({
        "id": P_ids[i], "nome": names[i], "mesa_fixa_esperada": mf, "mesas_encontradas": mesas_i,
        "ok": (mesas_i == [mf])
    })
fixed_check_df = pd.DataFrame(fixed_check)

print("Resumo:")
print("- must_together atendidos:", must_together_report['atendido'].sum(), "/", len(must_together_report))
print("- must_avoid violações   :", len(must_avoid_violations))
if not fixed_check_df.empty and not fixed_check_df['ok'].all():
    print("[ATENÇÃO] Algum fixo não ficou 100% na mesa esperada. Veja fixed_check_df.")


=== Diagnóstico das entradas ===
Participantes: 68
Mesas configuradas: 17 | Assentos/mesa: 4 | Vagas totais por rodada: 68
Pares must_together: 25 | pares must_avoid: 9
- Participantes fixos por mesa:
  Mesa 0: Caroline Scorvo
  Mesa 1: Cleiton Vicente
  Mesa 2: Fabiano Souza Ramos
  Mesa 3: Márcia Fonseca Borges
  Mesa 4: Nanci Toledo
  Mesa 5: Tiago Oliveira
- Nenhum gargalo teórico de must_together detectado.
- Participantes com mais restrições must_avoid:
  Cidinha Lins: evita 2 participante(s)
  Tiago  Vasconcelos: evita 2 participante(s)
  Ana Claudia Soares: evita 1 participante(s)
  Enaldye Soares: evita 1 participante(s)
  Andre Serra Martinez: evita 1 participante(s)
=== Fim do diagnóstico ===

[Solver] Status: FEASIBLE | Objetivo: -332795.0 | Bound: -1897210.0


C:\Users\felip\AppData\Local\Temp\ipykernel_4868\4008681990.py:483: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(to_wide)


UndefinedVariableError: name 'i' is not defined

## Validação Inicial (Pré segundo otimizador)

In [ ]:
seat_cols = [c for c in schedule_wide.columns if str(c).lower().startswith('p')]
pairs_meet_counts = {}

for _, row in schedule_wide.iterrows():
    pessoas = [str(x).strip() for x in row[seat_cols].tolist() if pd.notna(x) and str(x).strip()!='']
    for a, b in combinations(sorted(pessoas), 2):
        k = frozenset((a, b))
        pairs_meet_counts[k] = pairs_meet_counts.get(k, 0) + 1

def _find_col(d, *cands):
    cols = {c.lower(): c for c in d.columns}
    for c in cands:
        if c is None: 
            continue
        k = cols.get(c.lower())
        if k: 
            return k
    return None

# Tentar pegar nomes direto
mt_na = _find_col(must_together, 'nome_a', 'nome a', 'nome_a ')
mt_nb = _find_col(must_together, 'nome_b', 'nome b', 'nome_b ')
ma_na = _find_col(must_avoid,    'nome_a', 'nome a', 'nome_a ')
ma_nb = _find_col(must_avoid,    'nome_b', 'nome b', 'nome_b ')

# Se não tem nomes, tenta mapear por id usando participants
if (mt_na is None or mt_nb is None) or (ma_na is None or ma_nb is None):
    # descobrir colunas de id nas tabelas de regra
    mt_ia = _find_col(must_together, 'id_a', 'ida', 'id a')
    mt_ib = _find_col(must_together, 'id_b', 'idb', 'id b')
    ma_ia = _find_col(must_avoid,    'id_a', 'ida', 'id a')
    ma_ib = _find_col(must_avoid,    'id_b', 'idb', 'id b')

    # descobrir colunas em participants
    p_id  = _find_col(participants, 'id')
    p_nm  = _find_col(participants, 'nome_pessoa', 'nome', 'pessoa', 'nome completo')

    id2name = {}
    if p_id and p_nm:
        for _, r in participants[[p_id, p_nm]].iterrows():
            id2name[r[p_id]] = str(r[p_nm]).strip()

    # criar colunas de nome nas regras, se faltarem
    if mt_na is None and mt_ia and id2name:
        must_together['nome_a'] = must_together[mt_ia].map(id2name).fillna('').astype(str)
        mt_na = 'nome_a'
    if mt_nb is None and mt_ib and id2name:
        must_together['nome_b'] = must_together[mt_ib].map(id2name).fillna('').astype(str)
        mt_nb = 'nome_b'
    if ma_na is None and ma_ia and id2name:
        must_avoid['nome_a'] = must_avoid[ma_ia].map(id2name).fillna('').astype(str)
        ma_na = 'nome_a'
    if ma_nb is None and ma_ib and id2name:
        must_avoid['nome_b'] = must_avoid[ma_ib].map(id2name).fillna('').astype(str)
        ma_nb = 'nome_b'

# Último fallback: se ainda falta nome, cria vazio para evitar crash
if mt_na is None: 
    must_together['nome_a'] = ''
    mt_na = 'nome_a'
if mt_nb is None: 
    must_together['nome_b'] = ''
    mt_nb = 'nome_b'
if ma_na is None: 
    must_avoid['nome_a'] = ''
    ma_na = 'nome_a'
if ma_nb is None: 
    must_avoid['nome_b'] = ''
    ma_nb = 'nome_b'

must_together['_a'] = must_together[mt_na].astype(str).str.strip()
must_together['_b'] = must_together[mt_nb].astype(str).str.strip()
must_avoid['_a']    = must_avoid[ma_na].astype(str).str.strip()
must_avoid['_b']    = must_avoid[ma_nb].astype(str).str.strip()

# Remover linhas sem nomes válidos
must_together = must_together[(must_together['_a']!='') & (must_together['_b']!='')]
must_avoid    = must_avoid[(must_avoid['_a']!='') & (must_avoid['_b']!='')]

# =======================
# 3) MUST_TOGETHER — contagens e não atendidos (robusto p/ vazio)
# =======================
mt_counts = []
for _, r in must_together.iterrows():
    a = r['_a']; b = r['_b']
    k = frozenset((min(a,b), max(a,b)))
    c = pairs_meet_counts.get(k, 0)
    mt_counts.append({'nome_a': a, 'nome_b': b, 'vezes_encontraram': int(c)})

# Se vazio, garante colunas
if len(mt_counts) == 0:
    mt_counts_df = pd.DataFrame(columns=['nome_a','nome_b','vezes_encontraram'])
else:
    mt_counts_df = pd.DataFrame(mt_counts)
    mt_counts_df = mt_counts_df.sort_values(
        ['vezes_encontraram','nome_a','nome_b'],
        ascending=[False, True, True]
    ).reset_index(drop=True)

mt_nao_atendidos_df = mt_counts_df[mt_counts_df.get('vezes_encontraram', pd.Series(dtype=int))==0].reset_index(drop=True)

# =======================
# 4) MUST_AVOID — violações (robusto p/ vazio)
# =======================
ma_viol = []
for _, r in must_avoid.iterrows():
    a = r['_a']; b = r['_b']
    k = frozenset((min(a,b), max(a,b)))
    c = pairs_meet_counts.get(k, 0)
    if c > 0:
        ma_viol.append({'nome_a': a, 'nome_b': b, 'vezes_violadas': int(c)})

# Se vazio, garante colunas
if len(ma_viol) == 0:
    must_avoid_violations_df = pd.DataFrame(columns=['nome_a','nome_b','vezes_violadas'])
else:
    must_avoid_violations_df = pd.DataFrame(ma_viol)
    must_avoid_violations_df = must_avoid_violations_df.sort_values(
        ['vezes_violadas','nome_a','nome_b'],
        ascending=[False, True, True]
    ).reset_index(drop=True)

# =======================
# 5) Saída/resumo
# =======================
print("\n=== Verificação de Regras — Resumo ===")
print(f"Must_together (total de pares): {len(mt_counts_df)}")
print(f"  - Atendidos (>=1 encontro): {int((mt_counts_df['vezes_encontraram']>0).sum()) if not mt_counts_df.empty else 0}")
print(f"  - NÃO atendidos (0 encontro): {int((mt_counts_df['vezes_encontraram']==0).sum()) if not mt_counts_df.empty else 0}")
print(f"Must_avoid violações: {len(must_avoid_violations_df)}")

print("\n-- Pares must_together NÃO ATENDIDOS --")
print(mt_nao_atendidos_df.to_string(index=False) if not mt_nao_atendidos_df.empty else "(nenhum)")

print("\n-- Contagem de encontros por par (must_together) --")
print(mt_counts_df.to_string(index=False) if not mt_counts_df.empty else "(nenhum par listado)")

print("\n-- Violações must_avoid (encontros indevidos) --")
print(must_avoid_violations_df.to_string(index=False) if not must_avoid_violations_df.empty else "(nenhuma)")


=== Verificação de Regras — Resumo ===
Must_together (total de pares): 25
  - Atendidos (>=1 encontro): 17
  - NÃO atendidos (0 encontro): 8
Must_avoid violações: 0

-- Pares must_together NÃO ATENDIDOS --
                    nome_a                     nome_b  vezes_encontraram
                Ana Pessoa Carla Vanessa Romeo luongo                  0
                Ana Pessoa     Marcelo Henrique Alves                  0
Carla Vanessa Romeo luongo          Luciane Giesteira                  0
            Felipe Andrade          Maria Vasconcelos                  0
              Karina Fenix           Giselle Bontempo                  0
             Mariana Gobbo         Ana Claudia Soares                  0
             Mariana Gobbo             Tiago Oliveira                  0
             Walter Junior            Caroline Scorvo                  0

-- Contagem de encontros por par (must_together) --
                    nome_a                     nome_b  vezes_encontraram
          

,rodada,mesa,p1,p2,p3,p4
0,0,0,Caroline Scorvo,Gregor Osipoff,Jorge Luiz,Luciane Giesteira
1,0,1,Beth Souza,Cleiton Vicente,Tiago Vasconcelos,Willander Dos Reis Silva
2,0,2,Enaldye Soares,Fabiano Souza Ramos,Paulo Nistal,Ricardo Araujo
3,0,3,Brenda Almeida,Cidinha Lins,Márcia Fonseca Borges,Priscila Gambeta
4,0,4,Ailton Nunes,Marcelo Henrique Alves,Nanci Toledo,Silvana Zonatto
...,...,...,...,...,...,...
63,3,12,Ailton Nunes,Andre Serra Martinez,Fabiana Loureiro,Rosana Matos
64,3,13,Ana Pessoa,Fabiane Scandura,Katia Rocumback,Vinicius Terralheiro
65,3,14,Ana Claudia Soares,Christina Coutinho,Marcelo Henrique Alves,Walter Junior
66,3,15,Charlene Inacio,Daniela Silvestre da Silva,Luciane Giesteira,Silene Silva


## Segundo Otimizador (Post Process)

In [ ]:
seat_cols = [c for c in schedule_wide.columns if str(c).lower().startswith('p')]
R = sorted(schedule_wide['rodada'].unique().tolist())
T = sorted(schedule_wide['mesa'].unique().tolist())

# Mapeia nome -> (fixo, mesa_fixa) se schedule_df existir; senão, assume não fixo
nome_to_fixo = {}
nome_to_mesa_fixa = {}
if 'schedule_df' in globals():
    tmp_fix = schedule_df[['nome_pessoa','fixo','mesa_fixa']].drop_duplicates()
    for _, rr in tmp_fix.iterrows():
        nm = str(rr['nome_pessoa']).strip()
        nome_to_fixo[nm] = int(rr.get('fixo', 0)) if pd.notna(rr.get('fixo', np.nan)) else 0
        nome_to_mesa_fixa[nm] = int(rr['mesa_fixa']) if pd.notna(rr.get('mesa_fixa', np.nan)) else None

# Conjunto must_avoid por nome (bidirecional)
def _col(df, *cands):
    cc = {c.lower(): c for c in df.columns}
    for c in cands:
        if c and c.lower() in cc: return cc[c.lower()]
    return None

ma_na = _col(must_avoid,'nome_a','nome a')
ma_nb = _col(must_avoid,'nome_b','nome b')
if ma_na is None or ma_nb is None:
    ma_na = _col(must_avoid,'id_a','ida','id a'); ma_nb = _col(must_avoid,'id_b','idb','id b')
    p_id = _col(participants,'id'); p_nm = _col(participants,'nome_pessoa','nome','pessoa','nome completo')
    id2name = {}
    if p_id and p_nm:
        for _, rr in participants[[p_id,p_nm]].iterrows():
            id2name[rr[p_id]] = str(rr[p_nm]).strip()
    if ma_na and ma_nb and id2name:
        must_avoid['nome_a'] = must_avoid[ma_na].map(id2name).fillna('').astype(str)
        must_avoid['nome_b'] = must_avoid[ma_nb].map(id2name).fillna('').astype(str)
ma_na = _col(must_avoid,'nome_a'); ma_nb = _col(must_avoid,'nome_b')
ma_pairs = set()
if ma_na and ma_nb:
    for _, rr in must_avoid.iterrows():
        a = str(rr[ma_na]).strip(); b = str(rr[ma_nb]).strip()
        if a and b and a != b:
            ma_pairs.add(tuple(sorted([a,b])))

# must_together por nome (mapeia de id->nome se precisar)
mt_na = _col(must_together,'nome_a','nome a'); mt_nb = _col(must_together,'nome_b','nome b')
if mt_na is None or mt_nb is None:
    mt_ia = _col(must_together,'id_a','ida','id a'); mt_ib = _col(must_together,'id_b','idb','id b')
    p_id = _col(participants,'id'); p_nm = _col(participants,'nome_pessoa','nome','pessoa','nome completo')
    id2name = {}
    if p_id and p_nm:
        for _, rr in participants[[p_id,p_nm]].iterrows():
            id2name[rr[p_id]] = str(rr[p_nm]).strip()
    if mt_ia and mt_ib and id2name:
        must_together['nome_a'] = must_together[mt_ia].map(id2name).fillna('').astype(str)
        must_together['nome_b'] = must_together[mt_ib].map(id2name).fillna('').astype(str)
mt_na = _col(must_together,'nome_a'); mt_nb = _col(must_together,'nome_b')
mt_list = []
if mt_na and mt_nb:
    for _, rr in must_together.iterrows():
        a = str(rr[mt_na]).strip(); b = str(rr[mt_nb]).strip()
        if a and b and a != b:
            mt_list.append(tuple(sorted([a,b])))

# =========================
# 1) Estruturas do cronograma para permitir trocas
# =========================
# members[(r,t)] -> lista de nomes (ordenada para estabilidade visual)
members = {}
for _, row in schedule_wide.iterrows():
    r = int(row['rodada']); t = int(row['mesa'])
    lista = [str(x).strip() for x in row[seat_cols].tolist() if pd.notna(x) and str(x).strip()!='']
    members[(r,t)] = sorted(lista)

# person_pos[(r, nome)] -> mesa
person_pos = {}
for (r,t), lst in members.items():
    for nm in lst:
        person_pos[(r,nm)] = t

# person_tables[nome] -> set de mesas já usadas (para manter "não repetir mesa" p/ não fixos)
person_tables = {}
for (r,t), lst in members.items():
    for nm in lst:
        person_tables.setdefault(nm, set()).add(t)

# =========================
# 2) Contagem atual de encontros por par (para guiar swaps)
# =========================
meet_counts = {}
for r in R:
    for t in T:
        lst = members.get((r,t), [])
        for a, b in combinations(sorted(lst), 2):
            k = tuple(sorted([a,b]))
            meet_counts[k] = meet_counts.get(k, 0) + 1

# =========================
# 3) Alvo 1 — Atender pares must_together que estão com 0 encontros
#    Estratégia: por cada par (a,b) sem encontro, procurar uma rodada r
#    e trocar alguém da mesa de a ou b para juntá-los, respeitando fixos e must_avoid.
# =========================
for (a,b) in mt_list:
    if meet_counts.get((a,b), 0) > 0:
        continue  # já atendido
    # tentar em todas as rodadas, greedy
    for r in R:
        # se alguém não está nesta rodada, pula (em teoria todos estão)
        if (r,a) not in person_pos or (r,b) not in person_pos:
            continue
        t_a = person_pos[(r,a)]
        t_b = person_pos[(r,b)]
        if t_a == t_b:
            continue  # já juntos nesta rodada (raro, mas check)
        # preferir mover não-fixo; tentar trazer b -> t_a
        prefer_move = b
        target_t = t_a
        other_t = t_b
        # se b é fixo aqui, tenta o inverso (trazer a -> t_b)
        if nome_to_fixo.get(b, 0)==1:
            prefer_move = a; target_t = t_b; other_t = t_a
        mover = prefer_move
        stay = a if mover==b else b

        # se mover é fixo na rodada (ou tem mesa_fixa diferente), não pode
        if nome_to_fixo.get(mover, 0)==1:
            continue
        # respeitar "não repetir mesa" do mover:
        if target_t in person_tables.get(mover, set()):
            continue
        # checar must_avoid com alvo: mover deve poder sentar com todo mundo da mesa target_t
        ok_target = True
        for x in members[(r, target_t)]:
            if tuple(sorted([mover, x])) in ma_pairs:
                ok_target = False; break
        if not ok_target:
            continue

        # precisamos trocar 1×1: escolher alguém da mesa target_t para ir para other_t
        swapped = False
        for cand in members[(r, target_t)]:
            if cand == stay:
                continue
            # não trocar fixo
            if nome_to_fixo.get(cand, 0)==1:
                continue
            # cand pode ir para other_t sem violar must_avoid?
            ok_other = True
            for y in members[(r, other_t)]:
                if tuple(sorted([cand, y])) in ma_pairs:
                    ok_other = False; break
            if not ok_other:
                continue
            # cand não pode repetir mesa (se não for fixo)
            if other_t in person_tables.get(cand, set()):
                continue
            # não causar violação de fixo de mesa_fixa (se houver campo)
            mf_mover = nome_to_mesa_fixa.get(mover, None)
            mf_cand  = nome_to_mesa_fixa.get(cand, None)
            if mf_mover is not None and mf_mover != target_t:
                continue
            if mf_cand is not None and mf_cand != other_t:
                continue

            # Executa swap: mover sai de other_t -> target_t; cand sai de target_t -> other_t
            members[(r, other_t)].remove(mover)
            members[(r, target_t)].append(mover)
            members[(r, target_t)].remove(cand)
            members[(r, other_t)].append(cand)
            members[(r, target_t)] = sorted(members[(r, target_t)])
            members[(r, other_t)]  = sorted(members[(r, other_t)])

            # atualiza person_pos / person_tables
            person_pos[(r, mover)] = target_t
            person_pos[(r, cand)]  = other_t
            person_tables.setdefault(mover, set()).add(target_t)
            # remover mesa antiga do mover (other_t) segue no set; manter histórico é ok
            person_tables.setdefault(cand, set()).add(other_t)

            # atualiza meet_counts apenas desta rodada
            for x in members[(r, target_t)]:
                if x == mover: continue
                k = tuple(sorted([x, mover])); meet_counts[k] = meet_counts.get(k, 0) + 1
                if x == cand:
                    # antes eram colegas de mesa; já ajustado pela remoção/adição abaixo
                    pass
            for x in members[(r, other_t)]:
                if x == cand: continue
                k = tuple(sorted([x, cand])); meet_counts[k] = meet_counts.get(k, 0) + 1
            # Remover contagens antigas que deixaram de coexistir nesta rodada:
            # mover deixou de encontrar a turma de other_t; cand deixou de encontrar a turma de target_t
            # (fazendo decrementos seguros)
            for x in members[(r, other_t)]:
                k = tuple(sorted([x, mover]))
                if k in meet_counts and (x != cand):  # mover agora está em outra mesa
                    # ele não encontra mais x nesta rodada; mas atenção: x lista aqui já inclui cand pós-swap
                    pass
            for x in members[(r, target_t)]:
                k = tuple(sorted([x, cand]))
                if k in meet_counts and (x != mover):
                    pass
            # Para simplicidade, recomputamos meet_counts da rodada r inteira
            for pair in list(meet_counts.keys()):
                if pair[0]==pair[1]:
                    del meet_counts[pair]
            # limpa pares da rodada r
            # (mais simples: recalcular todos encontros da rodada r)
            for pair in list(meet_counts.keys()):
                # nada: vamos recontar logo abaixo apagando apenas pares que envolvem nomes desta rodada é complexo; reconta tudo da rodada r:
                pass
            # reconta da rodada r:
            for pair in list(meet_counts.keys()):
                # marcaremos para recontagem completa depois; mais fácil: zera e reconta tudo uma vez ao final deste swap
                pass
            meet_counts = {}
            for r2 in R:
                for t2 in T:
                    lst2 = members.get((r2,t2), [])
                    for u,v in combinations(sorted(lst2), 2):
                        kk = tuple(sorted([u,v]))
                        meet_counts[kk] = meet_counts.get(kk, 0) + 1

            swapped = True
            break
        if swapped:
            break  # segue para próximo par unmet

# =========================
# 4) Alvo 2 — Reduzir pares com mais de 1 encontro (se possível)
#    Estratégia: para k=(a,b) com contagem>1, tentar separar 1 ocorrência via swap simples
# =========================
dups = [k for k,c in meet_counts.items() if c > 1]
for (a,b) in sorted(dups):
    # procurar uma rodada onde a e b estão juntos e tentar separar
    done = False
    for r in R:
        if (r,a) not in person_pos or (r,b) not in person_pos: 
            continue
        if person_pos[(r,a)] != person_pos[(r,b)]:
            continue
        t = person_pos[(r,a)]
        # tentamos mover b para outra mesa t2 nesta rodada via swap com alguém de t2
        # preferir mover quem não for fixo
        mover = b if nome_to_fixo.get(b,0)==0 else (a if nome_to_fixo.get(a,0)==0 else None)
        if mover is None:
            continue
        # escolher mesa destino
        for t2 in T:
            if t2 == t:
                continue
            # evitar repetir mesa pro mover (se não fixo)
            if nome_to_fixo.get(mover,0)==0 and t2 in person_tables.get(mover,set()):
                continue
            # checar must_avoid com turma de t2
            ok_t2 = True
            for x in members[(r,t2)]:
                if tuple(sorted([mover,x])) in ma_pairs:
                    ok_t2 = False; break
            if not ok_t2:
                continue
            # escolher candidato em t2 pra vir pra t (swap)
            for cand in members[(r,t2)]:
                if nome_to_fixo.get(cand,0)==1:
                    continue
                # cand não pode violar must_avoid na mesa t
                ok_t = True
                for y in members[(r,t)]:
                    if y == mover: 
                        continue
                    if tuple(sorted([cand,y])) in ma_pairs:
                        ok_t = False; break
                if not ok_t:
                    continue
                # cand não pode repetir mesa (se não fixo)
                if nome_to_fixo.get(cand,0)==0 and t in person_tables.get(cand,set()):
                    continue
                # respeitar mesa_fixa, se houver
                mf_mover = nome_to_mesa_fixa.get(mover,None)
                mf_cand  = nome_to_mesa_fixa.get(cand,None)
                if mf_mover is not None and mf_mover != t2:
                    continue
                if mf_cand is not None and mf_cand != t:
                    continue

                # Executa swap
                members[(r,t)].remove(mover)
                members[(r,t2)].append(mover)
                members[(r,t2)].remove(cand)
                members[(r,t)].append(cand)
                members[(r,t)]  = sorted(members[(r,t)])
                members[(r,t2)] = sorted(members[(r,t2)])

                person_pos[(r,mover)] = t2
                person_pos[(r,cand)]  = t
                person_tables.setdefault(mover,set()).add(t2)
                person_tables.setdefault(cand,set()).add(t)

                # Reconta encontros globalmente (simples e seguro)
                meet_counts = {}
                for r2 in R:
                    for tt in T:
                        lst2 = members.get((r2,tt), [])
                        for u,v in combinations(sorted(lst2), 2):
                            kk = tuple(sorted([u,v]))
                            meet_counts[kk] = meet_counts.get(kk, 0) + 1
                done = True
                break
            if done:
                break
        if done:
            break

# =========================
# 5) Reconstruir schedule_wide atualizado
# =========================
new_rows = []
for r in R:
    for t in T:
        lst = members[(r,t)]
        row = {'rodada': r, 'mesa': t}
        for k, nm in enumerate(sorted(lst), start=1):
            row[f'p{k}'] = nm
        new_rows.append(row)
new_schedule_wide = pd.DataFrame(new_rows).sort_values(['rodada','mesa']).reset_index(drop=True)

# =========================
# 6) Relatório rápido pós-ajuste
# =========================
# Reconta must_together atendidos e duplicidades
pairs_meet_counts = {}
for _, row in new_schedule_wide.iterrows():
    lst = [str(x).strip() for x in row[seat_cols] if (x is not None and str(x).strip()!='')]
    for u,v in combinations(sorted(lst), 2):
        kk = tuple(sorted([u,v]))
        pairs_meet_counts[kk] = pairs_meet_counts.get(kk, 0) + 1

mt_zero = 0; mt_ok = 0; mt_gt1 = 0
for (a,b) in mt_list:
    c = pairs_meet_counts.get((a,b), 0)
    if c == 0: mt_zero += 1
    elif c == 1: mt_ok += 1
    else: mt_gt1 += 1

# must_avoid violações
ma_viol = 0
for (a,b) in ma_pairs:
    c = pairs_meet_counts.get((a,b), 0)
    if c > 0:
        ma_viol += 1

print("\n=== Pós-processamento — Resumo ===")
print(f"must_together: OK(=1)={mt_ok} | ZERO={mt_zero} | >1={mt_gt1}")
print(f"must_avoid violações: {ma_viol}")
print("\nPreview (5 linhas) do novo schedule_wide:")
print(new_schedule_wide.head().to_string(index=False))



=== Pós-processamento — Resumo ===
must_together: OK(=1)=21 | ZERO=3 | >1=1
must_avoid violações: 0

Preview (5 linhas) do novo schedule_wide:
 rodada  mesa              p1                  p2                    p3                       p4
      0     0 Caroline Scorvo          Jorge Luiz       Mateus Scandura         Valéria Donadeli
      0     1 Cleiton Vicente   Luciane Giesteira    Tiago  Vasconcelos Willander Dos Reis Silva
      0     2  Enaldye Soares Fabiano Souza Ramos          Paulo Nistal           Ricardo Araujo
      0     3  Brenda Almeida        Cidinha Lins Márcia Fonseca Borges         Priscila Gambeta
      0     4    Ailton Nunes  Daniela Fittipaldi          Nanci Toledo          Silvana Zonatto


## Validação Pós Segunda Otimização

In [ ]:
# =======================
# 1) Preparar nomes por mesa/rodada a partir de schedule_wide
# =======================
seat_cols = [c for c in new_schedule_wide.columns if str(c).lower().startswith('p')]
pairs_meet_counts = {}

for _, row in new_schedule_wide.iterrows():
    pessoas = [str(x).strip() for x in row[seat_cols].tolist() if pd.notna(x) and str(x).strip()!='']
    for a, b in combinations(sorted(pessoas), 2):
        k = frozenset((a, b))
        pairs_meet_counts[k] = pairs_meet_counts.get(k, 0) + 1

# =======================
# 2) Garantir que temos colunas de nome nas regras (caso venham por id)
# =======================
def _find_col(d, *cands):
    cols = {c.lower(): c for c in d.columns}
    for c in cands:
        if c is None: 
            continue
        k = cols.get(c.lower())
        if k: 
            return k
    return None

# Tentar pegar nomes direto
mt_na = _find_col(must_together, 'nome_a', 'nome a', 'nome_a ')
mt_nb = _find_col(must_together, 'nome_b', 'nome b', 'nome_b ')
ma_na = _find_col(must_avoid,    'nome_a', 'nome a', 'nome_a ')
ma_nb = _find_col(must_avoid,    'nome_b', 'nome b', 'nome_b ')

# Se não tem nomes, tenta mapear por id usando participants
if (mt_na is None or mt_nb is None) or (ma_na is None or ma_nb is None):
    # descobrir colunas de id nas tabelas de regra
    mt_ia = _find_col(must_together, 'id_a', 'ida', 'id a')
    mt_ib = _find_col(must_together, 'id_b', 'idb', 'id b')
    ma_ia = _find_col(must_avoid,    'id_a', 'ida', 'id a')
    ma_ib = _find_col(must_avoid,    'id_b', 'idb', 'id b')

    # descobrir colunas em participants
    p_id  = _find_col(participants, 'id')
    p_nm  = _find_col(participants, 'nome_pessoa', 'nome', 'pessoa', 'nome completo')

    id2name = {}
    if p_id and p_nm:
        for _, r in participants[[p_id, p_nm]].iterrows():
            id2name[r[p_id]] = str(r[p_nm]).strip()

    # criar colunas de nome nas regras, se faltarem
    if mt_na is None and mt_ia and id2name:
        must_together['nome_a'] = must_together[mt_ia].map(id2name).fillna('').astype(str)
        mt_na = 'nome_a'
    if mt_nb is None and mt_ib and id2name:
        must_together['nome_b'] = must_together[mt_ib].map(id2name).fillna('').astype(str)
        mt_nb = 'nome_b'
    if ma_na is None and ma_ia and id2name:
        must_avoid['nome_a'] = must_avoid[ma_ia].map(id2name).fillna('').astype(str)
        ma_na = 'nome_a'
    if ma_nb is None and ma_ib and id2name:
        must_avoid['nome_b'] = must_avoid[ma_ib].map(id2name).fillna('').astype(str)
        ma_nb = 'nome_b'

# Último fallback: se ainda falta nome, cria vazio para evitar crash
if mt_na is None: 
    must_together['nome_a'] = ''
    mt_na = 'nome_a'
if mt_nb is None: 
    must_together['nome_b'] = ''
    mt_nb = 'nome_b'
if ma_na is None: 
    must_avoid['nome_a'] = ''
    ma_na = 'nome_a'
if ma_nb is None: 
    must_avoid['nome_b'] = ''
    ma_nb = 'nome_b'

must_together['_a'] = must_together[mt_na].astype(str).str.strip()
must_together['_b'] = must_together[mt_nb].astype(str).str.strip()
must_avoid['_a']    = must_avoid[ma_na].astype(str).str.strip()
must_avoid['_b']    = must_avoid[ma_nb].astype(str).str.strip()

# Remover linhas sem nomes válidos
must_together = must_together[(must_together['_a']!='') & (must_together['_b']!='')]
must_avoid    = must_avoid[(must_avoid['_a']!='') & (must_avoid['_b']!='')]

# =======================
# 3) MUST_TOGETHER — contagens e não atendidos (robusto p/ vazio)
# =======================
mt_counts = []
for _, r in must_together.iterrows():
    a = r['_a']; b = r['_b']
    k = frozenset((min(a,b), max(a,b)))
    c = pairs_meet_counts.get(k, 0)
    mt_counts.append({'nome_a': a, 'nome_b': b, 'vezes_encontraram': int(c)})

# Se vazio, garante colunas
if len(mt_counts) == 0:
    mt_counts_df = pd.DataFrame(columns=['nome_a','nome_b','vezes_encontraram'])
else:
    mt_counts_df = pd.DataFrame(mt_counts)
    mt_counts_df = mt_counts_df.sort_values(
        ['vezes_encontraram','nome_a','nome_b'],
        ascending=[False, True, True]
    ).reset_index(drop=True)

mt_nao_atendidos_df = mt_counts_df[mt_counts_df.get('vezes_encontraram', pd.Series(dtype=int))==0].reset_index(drop=True)

# =======================
# 4) MUST_AVOID — violações (robusto p/ vazio)
# =======================
ma_viol = []
for _, r in must_avoid.iterrows():
    a = r['_a']; b = r['_b']
    k = frozenset((min(a,b), max(a,b)))
    c = pairs_meet_counts.get(k, 0)
    if c > 0:
        ma_viol.append({'nome_a': a, 'nome_b': b, 'vezes_violadas': int(c)})

# Se vazio, garante colunas
if len(ma_viol) == 0:
    must_avoid_violations_df = pd.DataFrame(columns=['nome_a','nome_b','vezes_violadas'])
else:
    must_avoid_violations_df = pd.DataFrame(ma_viol)
    must_avoid_violations_df = must_avoid_violations_df.sort_values(
        ['vezes_violadas','nome_a','nome_b'],
        ascending=[False, True, True]
    ).reset_index(drop=True)

# =======================
# 5) Saída/resumo
# =======================
print("\n=== Verificação de Regras — Resumo ===")
print(f"Must_together (total de pares): {len(mt_counts_df)}")
print(f"  - Atendidos (>=1 encontro): {int((mt_counts_df['vezes_encontraram']>0).sum()) if not mt_counts_df.empty else 0}")
print(f"  - NÃO atendidos (0 encontro): {int((mt_counts_df['vezes_encontraram']==0).sum()) if not mt_counts_df.empty else 0}")
print(f"Must_avoid violações: {len(must_avoid_violations_df)}")

print("\n-- Pares must_together NÃO ATENDIDOS --")
print(mt_nao_atendidos_df.to_string(index=False) if not mt_nao_atendidos_df.empty else "(nenhum)")

print("\n-- Contagem de encontros por par (must_together) --")
print(mt_counts_df.to_string(index=False) if not mt_counts_df.empty else "(nenhum par listado)")

print("\n-- Violações must_avoid (encontros indevidos) --")
print(must_avoid_violations_df.to_string(index=False) if not must_avoid_violations_df.empty else "(nenhuma)")
new_schedule_wide


=== Verificação de Regras — Resumo ===
Must_together (total de pares): 25
  - Atendidos (>=1 encontro): 22
  - NÃO atendidos (0 encontro): 3
Must_avoid violações: 0

-- Pares must_together NÃO ATENDIDOS --
                    nome_a            nome_b  vezes_encontraram
Carla Vanessa Romeo luongo Luciane Giesteira                  0
           Caroline Scorvo Luciane Giesteira                  0
             Walter Junior   Caroline Scorvo                  0

-- Contagem de encontros por par (must_together) --
                    nome_a                     nome_b  vezes_encontraram
           Giovanna Torres              Rosana Sallum                  2
                Ana Pessoa       Andre Serra Martinez                  1
                Ana Pessoa Carla Vanessa Romeo luongo                  1
                Ana Pessoa            Charlene Inacio                  1
                Ana Pessoa     Marcelo Henrique Alves                  1
                Ana Pessoa          Maria Vasc

,rodada,mesa,p1,p2,p3,p4
0,0,0,Caroline Scorvo,Jorge Luiz,Mateus Scandura,Valéria Donadeli
1,0,1,Cleiton Vicente,Luciane Giesteira,Tiago Vasconcelos,Willander Dos Reis Silva
2,0,2,Enaldye Soares,Fabiano Souza Ramos,Paulo Nistal,Ricardo Araujo
3,0,3,Brenda Almeida,Cidinha Lins,Márcia Fonseca Borges,Priscila Gambeta
4,0,4,Ailton Nunes,Daniela Fittipaldi,Nanci Toledo,Silvana Zonatto
...,...,...,...,...,...,...
63,3,12,Ailton Nunes,Andre Serra Martinez,Fabiana Loureiro,Rosana Matos
64,3,13,Ana Pessoa,Fabiane Scandura,Katia Rocumback,Vinicius Terralheiro
65,3,14,Ana Claudia Soares,Christina Coutinho,Marcelo Henrique Alves,Walter Junior
66,3,15,Charlene Inacio,Daniela Silvestre da Silva,Luciane Giesteira,Silene Silva


## Validações Gerais

In [6]:
# Validando encontros duplicados

schedule_wide[ (schedule_wide['p1'] == schedule_wide['p2']) |
    (schedule_wide['p1'] == schedule_wide['p3']) |
    (schedule_wide['p1'] == schedule_wide['p4']) |
    (schedule_wide['p2'] == schedule_wide['p3']) |
    (schedule_wide['p2'] == schedule_wide['p4']) |
    (schedule_wide['p3'] == schedule_wide['p4'])
]

,rodada,mesa,p1,p2,p3,p4


## Formatação Desejada Tabela Final

In [ ]:
# Gera df_final_premium (fixos) e df_final_standard (móveis)
# Fonte da verdade para FIXO = schedule_wide (mesma mesa nas 4 rodadas)
# Usa participants só para preencher EMPRESA. Não depende de Fixo/Mesa_Fixa do participants.

# aceita tanto schedule_wide quanto schedue_wide (typo)
if 'schedule_wide' not in globals() and 'schedue_wide' in globals():
    new_schedule_wide = new_schedue_wide

def _strip(s):
    if pd.isna(s): return ""
    s = unicodedata.normalize("NFKD", str(s).strip())
    return "".join(c for c in s if not unicodedata.combining(c))

def _det(df, *names):
    m = {c.lower(): c for c in df.columns}
    for n in names:
        if n and n.lower() in m:
            return m[n.lower()]
    return None

# 1) new_schedule_wide -> long
id_cols   = [c for c in new_schedule_wide.columns if c.lower() in {"rodada","mesa"}]
seat_cols = [c for c in new_schedule_wide.columns if c.lower().startswith("p")]
sw_long = (
    new_schedule_wide.melt(id_vars=id_cols, value_vars=seat_cols,
                       var_name="seat", value_name="nome_pessoa")
    .dropna(subset=["nome_pessoa"])
    .copy()
)
sw_long["mesa_label"]   = (sw_long["mesa"].astype(int) + 1).map(lambda x: f"MESA {x}")
sw_long["rodada_label"] = (sw_long["rodada"].astype(int) + 1).map(lambda x: f"{x}ª RODADA")

# 2) FIXO pelo schedule: quem ficou na MESMA mesa em todas as rodadas
mesas_dist = sw_long.groupby("nome_pessoa")["mesa"].nunique()
fixed_by_schedule = mesas_dist.eq(1).astype(int).rename("FIXO").reset_index()

# mesa fixa (rótulo) derivada do schedule
mesa_fixa_num = (
    sw_long.groupby("nome_pessoa")["mesa"]
           .agg(lambda s: s.iloc[0] if s.nunique()==1 else np.nan)
           .rename("MESA_FIXA_NUM")
           .reset_index()
)
mesa_fixa_num["MESA_FIXA"] = mesa_fixa_num["MESA_FIXA_NUM"].apply(
    lambda v: f"MESA {int(v)+1}" if pd.notna(v) else np.nan
)

# 3) EMPRESA via participants (sem afetar FIXO)
c_nome    = _det(participants, "Nome_Pessoa","nome_pessoa","Nome","Pessoa","nome")
c_empresa = _det(participants, "Nome_Empresa","nome_empresa","Empresa","empresa")

parts = participants.copy()
parts["_nome_key"] = parts[c_nome].map(lambda s: _strip(s).lower())
emp_por_nome = (
    parts.groupby("_nome_key")[c_empresa]
         .apply(lambda s: next((x for x in s if pd.notna(x) and str(x).strip()!=''), ""))
         .rename("EMPRESA")
         .reset_index()
)

sw_long["_nome_key"] = sw_long["nome_pessoa"].map(lambda s: _strip(s).lower())

# 4) base por pessoa (merge correto pela _nome_key)
pessoas = (sw_long[["nome_pessoa","_nome_key"]].drop_duplicates()
           .merge(fixed_by_schedule, on="nome_pessoa", how="left")
           .merge(mesa_fixa_num, on="nome_pessoa", how="left")
           .merge(emp_por_nome, on="_nome_key", how="left")
           .drop(columns=["_nome_key"])
           .rename(columns={"nome_pessoa":"NOME"}))
pessoas["EMPRESA"] = pessoas["EMPRESA"].fillna("").astype(str)

# 5) rodada × mesa por pessoa
rodada_mesa = (sw_long
    .pivot_table(index="nome_pessoa", columns="rodada_label",
                 values="mesa_label", aggfunc="first")
    .reset_index()
    .rename(columns={"nome_pessoa":"NOME"})
)

# 6) consolidação e split
pivot = pessoas.merge(rodada_mesa, on="NOME", how="left")
rod_cols = [c for c in ["1ª RODADA","2ª RODADA","3ª RODADA","4ª RODADA"] if c in pivot.columns]
pivot = pivot[["NOME","EMPRESA","FIXO","MESA_FIXA"] + rod_cols]

df_final_premium  = pivot[pivot["FIXO"]==1].copy()
df_final_standard = pivot[pivot["FIXO"]==0].copy()

# ordenação
if not df_final_premium.empty:
    ord_ = df_final_premium["MESA_FIXA"].str.extract(r"(\d+)")[0].astype(float)
    df_final_premium = (df_final_premium
                        .assign(_ord=ord_).sort_values(["_ord","NOME"])
                        .drop(columns=["_ord"]))
df_final_standard = df_final_standard.sort_values("NOME")
df_final_standard = df_final_standard.drop(columns=["FIXO","MESA_FIXA"])


### Exportação

final_path = fr'C:\Users\felip\Documents\FELIPE\Netweaving_Conecta\Github\netweaving-conecta-otimizador\saida_final'
os.chdir(final_path)

df_final_standard.to_excel('df_final_standard2.xlsx', index=False)
df_final_premium.to_excel('df_final_premium2.xlsx', index=False)